In [5]:
import ctypes
import sys

# Stacks

## Stack Implementations

In [2]:
class Empty(Exception):
    pass

In [10]:
class ArrayStack:
    '''LIFO Stack implementation using python list as an underlying storage.'''
    
    def __init__(self):
        '''Create an empty stack.'''
        self._data = []
    
    def __len__(self):
        '''Return the number of elements in the stack.'''
        return len(self._data)
    
    def push(self, e):
        '''Add an element to the top of the stack.'''
        self._data.append(e)
    
    def pop(self):
        '''Remove and return the element from the top of the stack.'''
        if self.is_empty():
            raise Empty('Stack is empty.')
        return self._data.pop()

    def top(self):
        '''Return the top element of the stack.'''
        if self.is_empty():
            raise Empty('Stack is empty.')
        return self._data[-1]
    
    def is_empty(self):
        '''Return True if the stack is empty.'''
        return len(self._data) == 0
    

In [96]:
class ResizedArrayStack:
    '''LIFO Stack implementation using low level resized arrays.'''
    
    def __init__(self, size=1):
        '''Create an empty stack.'''
        self._data = self._make_array(size)
        self._size = 0
        self._capacity = size
    
    def __len__(self):
        '''Return the number of elements in the stack.'''
        return self._size
    
    def push(self, e):
        '''Add an element to the top of the stack.'''
        if self._size == self._capacity:
            self._resize(self._capacity * 2)
        self._data[self._size] = e
        self._size += 1
    
    def pop(self):
        '''Remove and return the element from the top of the stack.'''
        if self.is_empty():
            raise Empty('Stack is empty.')
        e = self._data[self._size - 1]
        self._data[self._size -1] = None
        self._size -= 1
        if 0 < self._size == self._capacity / 4:
            self._resize(self._capacity // 2)
        return e

    def top(self):
        '''Return the top element of the stack.'''
        if self.is_empty():
            raise Empty('Stack is empty.')
        return self._data[self._size - 1]
    
    def is_empty(self):
        '''Return True if the stack is empty.'''
        return self._size == 0
    
    def reverse(self):
        if self._size <= 1:
            return
        for i in range(self._size // 2):
            self._data[i], self._data[self._size - i - 1] = (self._data[self._size - i - 1],
                                                             self._data[i])
    
    def __iter__(self):
        self._position = self._size
        return self
    
    def __next__(self):
        '''Returns the next element in the stack or raise StopIteration error.'''
        if self._position == 0:
            raise StopIteration()
        e = self._data[self._position - 1]
        self._position -= 1
        return e

    def _resize(self, capacity):
        '''Resize internal array to new capacity c.'''
        B = self._make_array(capacity)
        for i in range(self._size):
            B[i] = self._data[i]
        self._data = B
        self._capacity = capacity
    
    def _make_array(self, capacity):
        '''Return an empty array with capacity c.'''
        return (capacity * ctypes.py_object)()
    

In [97]:
S = ResizedArrayStack()
S.push(5)
S.push(3)

In [98]:
len(S)

2

In [99]:
S.top()

3

In [100]:
S.pop()

3

In [101]:
S.top()

5

In [102]:
len(S)

1

In [103]:
for s in S:
    print(s)

5


In [104]:
S.push(10)
S.push(100)

In [105]:
for s in S:
    print(s)

100
10
5


In [106]:
vars(S)

{'_data': <__main__.py_object_Array_4 at 0x1073a4378>,
 '_size': 3,
 '_capacity': 4,
 '_position': 0}

In [107]:
S.reverse()

In [108]:
for s in S:
    print(s)

5
10
100


## Stack Applications

### Reversing lines in a file

In [122]:
with open('file.txt', 'r') as f:
    for line in f.readlines():
        print(line.rstrip())

Line1
Line2
Line3


In [123]:
S = ResizedArrayStack()
with open('file.txt', 'r') as f:
    for line in f.readlines():
        S.push(line.rstrip('\n'))

In [124]:
for line in S:
    print(line)

Line3
Line2
Line1


In [125]:
with open('reversed-file.txt', 'w') as f:
    while not S.is_empty():
        f.write(S.pop() + '\n')

In [126]:
with open('reversed-file.txt', 'r') as f:
    for line in f.readlines():
        print(line.rstrip())

Line3
Line2
Line1


### Matching Parentheses

In [128]:
def is_matched(expr):
    left = '({['
    right = ')}]'
    S = ResizedArrayStack()
    for char in expr:
        if char in left:
            S.push(char)
        elif char in right:
            if S.is_empty():
                return False
            if right.index(char) != left.index(S.pop()):
                return False
    return S.is_empty()

In [129]:
is_matched('()')

True

In [130]:
is_matched('({[]})')

True

In [131]:
is_matched('(')

False

In [132]:
is_matched('(()')

False

# Queues

## Queue Implementations

In [212]:
class ResizedArrayQueue:
    '''FIFO Queue implementation using low level resized arrays.'''
    
    def __init__(self, size=1):
        '''Create an empty queue.'''
        self._data = self._make_array(size)
        self._size = 0
        self._capacity = size
        self._first = 0
        self._last = 0
    
    def __len__(self):
        '''Return the number of elements in the queue.'''
        return self._size
    
    def enqueue(self, e):
        '''Add an element to the end of the queue.'''
        if self._size == self._capacity:
            self._resize(self._capacity * 2)
        
        self._data[self._last] = e
        self._size += 1
        self._last += 1
        
        # If we reached the end
        if self._last == self._capacity:
            self._last = 0
    
    def dequeue(self):
        '''Remove and return the element from the beginning of the queue.'''
        if self.is_empty():
            raise Empty('queue is empty.')
        
        e = self._data[self._first]
        self._data[self._first] = None
        self._size -= 1
        self._first += 1
        
        if 0 < self._size == self._capacity / 4:
            self._resize(self._capacity // 2)
        
        if self._first == self._capacity:
            self._first = 0

        return e

    def first(self):
        '''Return the first element of the queue.'''
        if self.is_empty():
            raise Empty('queue is empty.')
        return self._data[self._first]
    
    def is_empty(self):
        '''Return True if the queue is empty.'''
        return self._size == 0
    
    def reverse(self):
        if self._size <= 1:
            return
        for i in range(self._size // 2):
            self._data[i], self._data[self._size - i - 1] = (self._data[self._size - i - 1],
                                                             self._data[i])
    
    def __iter__(self):
        self._position = 0
        return self
    
    def __next__(self):
        '''Returns the next element in the queue or raise StopIteration error.'''
        if self._position == self._size:
            raise StopIteration()
        e = self._data[(self._position + self._first) % self._capacity]
        self._position += 1
        return e

    def _resize(self, capacity):
        '''Resize internal array to new capacity c.'''
        B = self._make_array(capacity)
        for i in range(self._size):
            print(i, self._first, self._last, self._size)
            B[i] = self._data[(self._first + i) % self._size]
        self._data = B
        self._capacity = capacity
        self._first = 0
        self._last = self._size
    
    def _make_array(self, capacity):
        '''Return an empty array with capacity c.'''
        return (capacity * ctypes.py_object)()
    

In [213]:
Q = ResizedArrayQueue()
Q.enqueue(5)
Q.enqueue(3)
len(Q), Q.first()

0 0 0 1


(2, 5)

In [214]:
Q.is_empty()

False

In [215]:
Q.enqueue(50)
Q.enqueue(500)
Q.enqueue(.5)

0 0 0 2
1 0 0 2
0 0 0 4
1 0 0 4
2 0 0 4
3 0 0 4


In [216]:
len(Q)

5

In [217]:
for q in Q:
    print(q)

5
3
50
500
0.5


In [218]:
Q.dequeue()

5

In [219]:
len(Q)

4

In [220]:
for q in Q:
    print(q)

3
50
500
0.5


In [221]:
Q.dequeue()

3

In [222]:
len(Q)

3

In [223]:
for q in Q:
    print(q)

50
500
0.5


# Deques

In [293]:
class ArrayDeque:
    '''Implementation of Double-Ended Queues (Deque) using Resized Arrays.'''
    
    def __init__(self, size=1):
        '''Create an empty deque.'''
        self._size = 0
        self._capacity = size
        self._data = self._make_array(self._capacity)
        self._first = 0
        self._last = 0

    def __len__(self):
        '''Return the number of elements in the queue.'''
        return self._size

    def is_empty(self):
        '''Return True if the queue is empty.'''
        return self._size == 0

    def first(self):
        '''Return the first element of the deque.'''
        return self._data[self._first]
    
    def last(self):
        '''Return the last element of the deque.'''
        return self._data[self._last]
    
    def add_first(self, e):
        '''Add element to the front of the deque.'''
        if self._size == self._capacity:
            self._resize(self._capacity * 2)
        
        self._first = (self._first - 1) % self._capacity
        self._data[self._first] = e
        self._size += 1
    
    def add_last(self, e):
        '''Add element to the back of the deque.'''
        if self._size == self._capacity:
            self._resize(self._capacity * 2)
        self._last = (self._last + 1) % self._capacity
        self._data[self._last] = e
        self._size += 1

    def remove_first(self):
        '''Remove and return the first element from the deque.'''
        if 0 < self._size == self._capacity / 4:
            self._resize(self._capacity / 2)

        e = self._data[self._first]
        self._data[self._first] = None
        self._first = (self._first + 1) % self._capacity
        self._size -= 1
        return e
    
    def remove_last(self):
        '''Remove and return the last element from the deque.'''
        if 0 < self._size == self._capacity / 4:
            self._resize(self._capacity / 2)
        
        e = self._data[self._last]
        self._data[self._last] = None
        self._last = (self._last - 1) % self._capacity
        self._size -= 1
        return e

    def get(self, i):
        '''Get the element at index i; otherwise return None'''
        if self._size == 0 or i >= self._size:
            return

        e = self._data[(self._first + i) % self._capacity]
        return e
    
    def __iter__(self):
        self._position = 0
        return self
    
    def __next__(self):
        '''Returns the next element in the deque or raise StopIteration error.'''
        if self._position == self._size:
            raise StopIteration()
        
        e = self._data[(self._position + self._first) % self._capacity]
        self._position += 1
        return e

    def _resize(self, capacity):
        '''Resize internal array to new capacity c.'''
        B = self._make_array(capacity)
        for i in range(self._size):
            B[i] = self._data[(self._first + i) % self._capacity]
        self._data = B
        self._capacity = capacity
        self._first = 0
        self._last = self._size - 1
    
    def _make_array(self, capacity):
        '''Return an empty array with capacity c.'''
        return (capacity * ctypes.py_object)()

In [294]:
D = ArrayDeque()

In [295]:
D.add_first(10)
D.add_first(1)
D.add_last(100)
len(D)

3

In [296]:
D.first()

1

In [297]:
D.last()

100

In [298]:
D._capacity

4

In [299]:
D._size

3

In [300]:
D.remove_first()

1

In [301]:
D.first()

10

In [302]:
D.last()

100

In [303]:
D._capacity

4

In [304]:
D._size

2

In [305]:
D.remove_last()

100

In [306]:
D.last(), D.first()

(10, 10)

In [307]:
D._size

1

In [308]:
D._capacity

4

In [309]:
D.add_first(3)
D.add_first(2)
D.add_last(6)

In [310]:
for e in D:
    print(e)

2
3
10
6


In [311]:
D.get(4)

In [312]:
D.get(2)

10